### Polecenie
1. Utworzyć korpus dokumentów na podstawie 5-10 książek pobranych ze strony [Wolne Lektury](https://wolnelektury.pl/)

2. Dla każdej z pozycji zwizualizować częstość występowania tokenów i skomentować zgodność z prawem Zipfa. Jako tokeny przyjmujemy tylko tokeny zbudowane wyłącznie z liter.<br>
Zastosować stoplistę  dla języka polskiego.

3. Na podstawie korpusu zbudowanego   z 5 dowolnych książek ściągniętych ze strony Wolne Lektury zbudować macierz tfidf.  

4. W oparciu o podobieństwo kosinusowe zbudować macierz odległości dokumentów. 

5. Przeprowadzić redukcję wymiarowości wymiarów macierzy tf-idf do 2 wymiarów  i zwizualizować położenie dokumentów na układzie współrzędnych.  

6. Skomentować spójność wyników z punktu 4 i 5. Które dokumenty są najbardziej podobne do siebie  w obydwu wariantach?

###Tworzenie korpusu

In [1]:
import urllib.request
def open_book(url):
  file = urllib.request.urlopen(url)
  file = file.read().decode('utf-8')
  return file[file.find("ISBN")+26:file.rfind("-----")]

Funkcja wykorzystująca bibliotekę <font color='gold'>***urllib***</font> w celu otworzenia plików txt ze strony internetowej. Dodatkowo funkcja przycina nieistotny początek i koniec witryny. 

In [2]:
urls = ["https://www.wolnelektury.pl/media/book/txt/brzydkie-kaczatko.txt", 
        "https://wolnelektury.pl/media/book/txt/calineczka.txt", 
        "https://wolnelektury.pl/media/book/txt/dziewczynka-z-zapalkami.txt",
        "https://wolnelektury.pl/media/book/txt/puszkin-bajka-o-rybaku-i-rybce.txt",
        "https://wolnelektury.pl/media/book/txt/pan-grubas.txt",
        "https://wolnelektury.pl/media/book/txt/lament-swietokrzyski.txt"]

corpus = []
for url in urls:
  corpus.append(open_book(url))

Dodawanie poszczególnych książek do korpusu.

###Prawo Zipfa

In [3]:
import string
punctuations = list(string.punctuation)
punctuations.extend(["!…", "–", "…", "«", "»", "„", '”', "”.", ").", "—", "».", "!”"])

Lista zawierająca znaki interpunkcyjne, rozszerzona o znaki występujące w dokumentach z korpusu.

In [4]:
from nltk import wordpunct_tokenize
def divide_to_tokens(vector):
  for token in wordpunct_tokenize(vector):
    if token not in punctuations:
      if token.lower() in d:
        amount = d[token.lower()]
        d[token.lower()] = amount+1
      else:
        d[token.lower()] = 1
  return d

Funkcja dzieląca wektor na tokeny dzięki funkcji <font color='gold'>***wordpunct_tokenize***</font> z biblioteki 
<font color='gold'>***nltk***</font> i wykluczająca interpunkcję. Dodatkowo w funkcji następuje przekształcenie do zapisu małymi literami. 

In [5]:
def print_tokens_frequency(d):
  d = dict(sorted(d.items(), key=lambda item: item[1], reverse=True))
  print("\033[1;34m" + 
        "{:<20} {:<12} {:<8} {:<8}".format("Word", "Frequency", "Rank", "f*r") + 
        "\033[0m")
  i = 1
  for k, v in d.items():
        print("{:<20} {:<12} {:<8} {:<8}".format(k, v, i, v*i))
        i+=1
        if i==31:
          break
  print("\n\n")

Funkcja wypisująca zawartość posortowanego malejąco słownika i licząca iloczyn liczby wystąpień słowa i pozycji słowa w rankingu. Ograniczyłam liczbę słów do pierwszych 30 dla większej przejrzystości wyników.

In [6]:
d = {}
for vector in corpus:
  d = divide_to_tokens(vector)
  print_tokens_frequency(d)
  d.clear()

Word                 Frequency    Rank     f*r     
i                    135          1        135     
się                  114          2        228     
nie                  73           3        219     
z                    66           4        264     
na                   61           5        305     
w                    52           6        312     
a                    43           7        301     
to                   43           8        344     
do                   31           9        279     
że                   29           10       290     
tak                  27           11       297     
jak                  25           12       300     
kaczę                25           13       325     
o                    22           14       308     
za                   17           15       255     
było                 16           16       256     
co                   16           17       272     
je                   16           18       288     
po          

Na podstawie przedstawionej powyżej tabeli widzimy, że prawo Zipfa jest spełnione. Częstość słowa jest odwrotnie proporcjonalna do rangi, zatem iloczyn rangi i częstości jest stały.

####Ze stoplistą

In [7]:
stoplist = urllib.request.urlopen("https://raw.githubusercontent.com/bieli/stopwords/master/polish.stopwords.txt")
stoplist = stoplist.read().decode('utf-8').splitlines()

Na podstawie pliku txt znalezionego w internecie tworzę stoplistę.

In [8]:
for vector in corpus:
  d = divide_to_tokens(vector)
  d = {k:v for k, v in d.items() if k not in stoplist}
  print_tokens_frequency(d)
  d.clear()

Word                 Frequency    Rank     f*r     
kaczę                25           1        25      
brzydkie             15           2        30      
kaczka               12           3        36      
rzekła               12           4        48      
kaczki               11           5        55      
wody                 10           6        60      
wodzie               9            7        63      
dzieci               9            8        72      
kaczątko             9            9        81      
dalej                9            10       90      
wielkie              8            11       88      
kwa                  8            12       96      
świat                8            13       104     
ptaki                8            14       112     
wreszcie             7            15       105     
zaczęły              7            16       112     
oczy                 7            17       119     
duże                 7            18       126     
wtem        

Stoplista nie zmienia prawdziwości prawa Zipfa, zmienia jednak diametralnie wypisywany wektor. Na przykład w pierwszym dokumencie widzimy, że z poprzednią tabelą pokrywają się jedynie 4 słowa - kaczę, brzydkie, kaczka, rzekła.

###Macierz Tf-Idf

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
# import numpy as np
tfidf = TfidfVectorizer(smooth_idf=False, stop_words=stoplist)
tfs = tfidf.fit_transform(corpus)
# np.set_printoptions(threshold=np.inf)
tfs.todense()

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.01517399, ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.02079116, 0.02079116, 0.        , ..., 0.02079116, 0.02079116,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.0748311 ]])

Z wykorzystaniem wcześniej przygotowanej stoplisty tworzę przy użyciu biblioteki <font color='gold'>***sklearn***</font> macierz tf-idf. (Zakomentowany kod pozwala wyświetlić macierz w całości.)

In [10]:
tfidf.get_feature_names()[2:22]

['abyśmy',
 'aksamitne',
 'aksamitnym',
 'anjele',
 'apetyt',
 'ażeby',
 'babcia',
 'babciu',
 'babie',
 'babka',
 'babki',
 'babo',
 'babuni',
 'babunia',
 'babunię',
 'babą',
 'bajać',
 'bajeczkę',
 'bajek',
 'bali']

Powyżej wypisane jest 20 przykładowych słów z macierzy.

###Podobieństwo cosinusowe

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_frequency = CountVectorizer(stop_words=stoplist).fit_transform(corpus)
tokens_frequency.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [1, 1, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 1]])

Tworzymy i wypisujemy macierz częstotliwości występowania słów z gotowej funkcji biblioteki <font color='gold'>***sklearn***</font>. Wykluczamy słowa ze naszej stoplisty.

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
from numpy import around
around(cosine_similarity(tokens_frequency), 3)

array([[1.   , 0.212, 0.123, 0.089, 0.098, 0.026],
       [0.212, 1.   , 0.186, 0.075, 0.068, 0.041],
       [0.123, 0.186, 1.   , 0.029, 0.025, 0.034],
       [0.089, 0.075, 0.029, 1.   , 0.028, 0.017],
       [0.098, 0.068, 0.025, 0.028, 1.   , 0.004],
       [0.026, 0.041, 0.034, 0.017, 0.004, 1.   ]])

Dzięki funkcji liczącej podobieństwo cosinusowe z biblioteki <font color='gold'>***sklearn***</font> tworzymy macierz odległości dokumentów. Zaokrąglamy do 3 miejsc po przecinku dzięki <font color='gold'>***numpy***</font>.

###PCA

In [13]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
visualization = pca.fit_transform(tfs.todense())
pca.components_

array([[ 0.00572882,  0.00572882, -0.00647634, ...,  0.00572882,
         0.00572882,  0.04805425],
       [ 0.00435271,  0.00435271, -0.00228295, ...,  0.00435271,
         0.00435271, -0.04148485]])

Dzięki gotowej funkcji PCA z biblioteki <font color='gold'>***sklearn***</font> redukujemy macierz tf-idf fo 2 wymiarów.

In [14]:
import plotly.express as px
names = ["Brzydkie kaczątko", "Calineczka", "Dziewczynka z zapałkami", "Puszkin", "Pan Grubas", "Lament Świętokrzyski"]
fig = px.scatter(x=visualization[:, 0], y=visualization[:, 1],
                 text=names,
                 width=700, height=500)
fig.update_traces(textposition="bottom center")
fig.update_xaxes(title="pca1",
                 range=[-0.7, 0.95])
fig.update_yaxes(title="pca2")
fig.show()

Na układzie współrzędnych widzimy położenie każdego z dokumentów.

###Wnioski

#####PCA
Na podstawie wykresu widzimy, że względem zmiennej PCA1 najbardziej zbliżone są powieści H.C.Andersena oraz osobno Bajki braci Grimm i Puszkina. Względem zmiennej PCA2 najwięcej wspólnego mają ze sobą bajki "Brzydkie kaczątko" i "Bajka o rybaku i rybce".

##### Podobieństwo cosinusowe
Na podstawie macierzy widzimy, że najbardziej podobne do siebie są "Calineczka" i "Brzydkie kaczątko". Podobieństwo występuję również miedzy pozostałymi baśniami H.C. Andersena.

#####Podsumowanie
Obie metody wskazują na podobieństwo baśni H.C. Andersena.